In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns
from matplotlib.ticker import ScalarFormatter, AutoMinorLocator

warnings.filterwarnings("ignore", message="Workbook contains no default style")

FILE_PATH = 'C:\\Users\\vladi\\.vscode\\Git\\SM_25\\SM_25\\cursova\\'

In [10]:
def get_verification_result(df):
    total_time = df['total_time']

    loader1_busy_time = df['Loader1_busyTime']
    loader2_busy_time = df['Loader2_busyTime']

    truck1_served = df["Truck1_served"]
    truck1_busy_time = df["Truck1_busyTime"]
    truck1_cooldownTime = df["Truck1_cooldownTime"]

    truck2_served = df["Truck2_served"]
    truck2_busy_time = df["Truck2_busyTime"]
    truck2_cooldownTime = df["Truck2_cooldownTime"]

    truck3_served = df["Truck3_served"]
    truck3_busy_time = df["Truck3_busyTime"]
    truck3_cooldownTime = df["Truck3_cooldownTime"]

    truck4_served = df["Truck4_served"]
    truck4_busy_time = df["Truck4_busyTime"]
    truck4_cooldownTime = df["Truck4_cooldownTime"]

    queue_served = df['Queue_served']
    queue_pair_avg_wt = df['Queue_pair_avg_wt']

    truck_served = truck1_served + truck2_served + truck3_served + truck4_served

    truck_utilization = (
        (loader1_busy_time + loader2_busy_time) + 
        (truck1_busy_time + truck2_busy_time + truck3_busy_time + truck4_busy_time) +
        (truck1_cooldownTime + truck2_cooldownTime + truck3_cooldownTime + truck4_cooldownTime)
    ) / total_time / 4

    productivity = truck_served / total_time
    processing_time = (
        (queue_pair_avg_wt * queue_served / 2) + 
        loader1_busy_time + 
        loader2_busy_time +
        truck1_busy_time +
        truck2_busy_time +
        truck3_busy_time +
        truck4_busy_time
    ) / truck_served


    df['Truck_served'] = truck_served
    df['Truck_utilization'] = truck_utilization
    df['Productivity'] = productivity
    df['ProcessingTime'] = processing_time

    df = df.groupby('total_time').mean().reset_index()

    return df[['Producer1_served', 
               'Queue_served',
               'Queue_pair_avg_sz',
               'Queue_pair_avg_wt',
               'Con0_throughput',
               'Con0_availability',
               'Loader1_utilization',
               'Loader1_served',
               'Loader2_utilization',
               'Loader2_served',
               'Truck_served',
               'Truck_utilization',
               'Productivity',
               'ProcessingTime'
            ]].copy()


#def get_experimental_result(df):
    

In [11]:
def get_verification():
    filename = 'verification'
    df_list = []

    for i in range(0, 8):
        full_path = f"{FILE_PATH}{filename}{i}.xlsx"

        df_v = pd.read_excel(full_path, engine="openpyxl")
        df_list.append(get_verification_result(df_v))

    final_df = pd.concat(df_list, ignore_index=True)
    return final_df

get_verification()

,Producer1_served,Queue_served,Queue_pair_avg_sz,Queue_pair_avg_wt,Con0_throughput,Con0_availability,Loader1_utilization,Loader1_served,Loader2_utilization,Loader2_served,Truck_served,Truck_utilization,Productivity,ProcessingTime
0,179.807,144.716,8.874489,180.048257,19.985045,0.650799,0.469048,35.815,0.421470,35.851,70.279,0.725149,0.048805,226.347095
1,359.532,146.568,52.894795,1046.547251,19.734601,0.625914,0.478242,36.508,0.427431,36.114,71.238,0.735978,0.049471,1117.988620
2,179.650,163.360,4.415931,78.769799,17.671586,0.566938,0.340640,40.827,0.475904,40.289,79.539,0.772467,0.055235,118.257634
3,179.733,159.574,5.405801,99.335154,18.109175,0.584627,0.523286,39.578,0.302797,39.648,77.659,0.760728,0.053930,139.989844
4,180.103,123.150,14.079973,334.499491,23.509751,0.773539,0.509221,30.564,0.464954,30.453,59.852,0.619610,0.041564,385.460951
5,179.856,151.322,7.328749,142.839027,19.124666,0.800551,0.496013,37.428,0.445027,37.560,74.128,0.619388,0.051478,175.502576
6,180.342,146.250,8.603523,172.672062,19.778683,0.675994,0.475446,36.192,0.426889,36.255,71.146,0.708860,0.049407,216.391821
7,180.375,150.880,7.560530,147.306192,19.179495,0.774104,0.492322,37.128,0.444329,37.611,73.332,0.639676,0.050925,192.626769
